## Choose the hyper parameter for KNN

In [3]:
import numpy as np
import pandas as pd
import helper
import KNNTest
import re
import pickle
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score,precision_score, recall_score
vowel = {'AA': 0, 'AW': 1, 'AY': 2, 'ER': 3, 'EY': 4, 'IY': 5, 'OW': 6, 'OY': 7, 'UW': 8, 'AE': 9, 'AH': 10, 'AO': 11, 'EH': 12, 'IH': 13, 'UH': 14}
consonant = {'AA': 0, 'AW': 1, 'AY': 2, 'ER': 3, 'EY': 4, 'IY': 5, 'OW': 6, 'OY': 7, 'UW': 8, 'AE': 9, 'AH': 10, 'AO': 11, 'EH': 12, 'IH': 13, 'UH': 14, 'P': 15, 'B': 16, 'CH': 17, 'D': 18, 'DH': 19, 'F': 20, 'G': 21, 'HH': 22, 'JH': 23, 'K': 24, 'L': 25, 'M': 26, 'N': 27, 'NG': 28, 'R': 29, 'S': 30, 'SH': 31, 'T': 32, 'TH': 33, 'V': 34, 'W': 35, 'Y': 36, 'Z': 37, 'ZH': 38}
raw_data = helper.read_data('./asset/training_data.txt')

In [4]:
final2, final3, final4 = KNNTest.train(raw_data, '')

In [10]:
final4[:2]

,Target,C1234,C234,C123,C34,C23,C12,C4,C3,C2,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
0,2,N AA N P OY Z AH N AH S,P OY Z AH N AH S,N AA N P OY Z AH,Z AH N AH S,P OY Z AH,N AA N P OY,N AH S,Z AH,P OY,...,1,0,0,0,0,0,0,0,0,0
1,2,L AA V EH K IY AH,V EH K IY AH,L AA V EH K IY,K IY AH,V EH K IY,L AA V EH,AH,K IY,V EH,...,0,0,0,0,0,0,0,0,0,1


In [7]:
print(final2.shape)
print(final3.shape)
print(final4.shape)

(27682, 263)
(15409, 361)
(4896, 460)


## Vowel 4

In [24]:
## choose k
train, test = train_test_split(final4, test_size = 0.1)
features = list(final4.columns)[11:]
combinations = list(final4.columns)[1:11]
train_X = train[features]
train_y = train['Target']
test_X = test[features]
test_y = test['Target']

In [26]:
#for i in range(10,20):
#    print ('k = ' + str(i))
#    neigh = KNeighborsClassifier(n_neighbors = i, weights='distance')
#    neigh.fit(train_X, train_y)
#    print('Training acc : ' + str(neigh.score(train_X, train_y)))
#    print('Testing acc : ' + str(neigh.score(test_X, test_y)))
#    pred = neigh.predict(test_X)
#    print('F1 : ' + str(f1_score(list(test_y), pred, average='macro')))

In [28]:
## Get average f1 before refine
## change dataset
## K = 15
f1 = 0
for round in range(1,6):
    print("Round : " + str(round))
    train, test = train_test_split(final4, test_size = 0.1)
    features = list(final4.columns)[11:]
    train_X = train[features]
    train_y = train['Target']
    test_X = test[features]
    test_y = test['Target']
    neigh = KNeighborsClassifier(n_neighbors = 15, weights='distance')
    neigh.fit(train_X, train_y)
    print('Training acc : ' + str(neigh.score(train_X, train_y)))
    print('Testing acc : ' + str(neigh.score(test_X, test_y)))
    pred = neigh.predict(test_X)
    curr_f1 = f1_score(list(test_y), pred, average='macro')
    f1 += curr_f1
    print('F1 : ' + str(curr_f1))
    print("= = = = = Round End = = = = =")
print("Average F1 : " + str(f1 * 1.0 / 5))

Round : 1
Training acc : 0.999546073536
Testing acc : 0.869387755102


/home/venturer/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1 : 0.634843321617
= = = = = Round End = = = = =
Round : 2
Training acc : 0.999546073536
Testing acc : 0.885714285714
F1 : 0.639249829906
= = = = = Round End = = = = =
Round : 3
Training acc : 0.999546073536
Testing acc : 0.875510204082
F1 : 0.644901237975
= = = = = Round End = = = = =
Round : 4
Training acc : 0.999546073536
Testing acc : 0.879591836735
F1 : 0.639187127268
= = = = = Round End = = = = =
Round : 5
Training acc : 0.999546073536
Testing acc : 0.859183673469
F1 : 0.632619007766
= = = = = Round End = = = = =
Average F1 : 0.638160104906


In [29]:
def get_refined_df(test_combination, df):
    df_features = ['C1234','C234','C123','C34','C23','C12','C4','C3','C2','C1']
    for i in range(len(df_features)):
        sub_df = df[df[df_features[i]].str.contains('^' + test_combination[i] + '$')]
        if sub_df.shape[0] > 0:
            # print("Pattern : " + df_features[i] + " , " + test_combination[i])
            return sub_df
    return df

In [31]:
## Get refined f1
def test_model(train, test, features):
    pred = []
    test_X_total = test[features]
    test_syllables_comb = test[syllables_comb]
    for i in range(test.shape[0]):
        curr_X = np.array(test_X_total.iloc[i]).reshape((1,-1))
        curr_syllables_comb = list(test_syllables_comb.iloc[i])
        
        sub_train = get_refined_df(curr_syllables_comb, train)
        train_X = sub_train[features]
        train_y = sub_train["Target"]
        
        if train_X.shape[0] < 15:
            neigh = KNeighborsClassifier(n_neighbors = train_X.shape[0], weights='distance')
            # neigh = KNeighborsClassifier(n_neighbors = 1)
        else:
            neigh = KNeighborsClassifier(n_neighbors = 15, weights='distance')
        neigh.fit(train_X, train_y)
        curr_pred = neigh.predict(curr_X)[0]
        pred.append(curr_pred)
    return pred

In [ ]:
#f1 = 0
#for i in range(10):
#    print("Round : " + str(i))
#    train, test = train_test_split(final4, test_size = 0.1)
#    features = list(final4.columns)[11:]
#    syllables_comb = list(final4.columns)[1:11]
#    test_y = list(test.Target)
#    pred = test_model(train, test, features)
#    curr_f1 = f1_score(test_y, pred, average='macro')
#    f1 += curr_f1
#    print('F1 : ' + str(curr_f1))
#print('Average F1 : ' + str(f1 * 1.0 / 5))

## Vowel 3

In [34]:
## choose k
train, test = train_test_split(final3, test_size = 0.1)
features = list(final3.columns)[7:]
combinations = list(final3.columns)[1:7]
train_X = train[features]
train_y = train['Target']
test_X = test[features]
test_y = test['Target']
for i in range(5,15):
    print ('k = ' + str(i))
    neigh = KNeighborsClassifier(n_neighbors = i, weights='distance')
    neigh.fit(train_X, train_y)
    print('Training acc : ' + str(neigh.score(train_X, train_y)))
    print('Testing acc : ' + str(neigh.score(test_X, test_y)))
    pred = neigh.predict(test_X)
    print('F1 : ' + str(f1_score(list(test_y), pred, average='macro')))

k = 5
Training acc : 0.998629939429
Testing acc : 0.882543802726
F1 : 0.753440131186
k = 6
Training acc : 0.998629939429
Testing acc : 0.883841661259
F1 : 0.750316693505
k = 7
Training acc : 0.998629939429
Testing acc : 0.887086307592
F1 : 0.748289437055
k = 8
Training acc : 0.998629939429
Testing acc : 0.889033095393
F1 : 0.755370799534
k = 9
Training acc : 0.998629939429
Testing acc : 0.889033095393
F1 : 0.751947040861
k = 10
Training acc : 0.998629939429
Testing acc : 0.894224529526
F1 : 0.76071770606
k = 11
Training acc : 0.998629939429
Testing acc : 0.891628812459
F1 : 0.756687299686
k = 12
Training acc : 0.998629939429
Testing acc : 0.89552238806
F1 : 0.76568676419
k = 13
Training acc : 0.998629939429
Testing acc : 0.89552238806
F1 : 0.766999856051
k = 14
Training acc : 0.998629939429
Testing acc : 0.892277741726
F1 : 0.763303022106


In [36]:
## Get average f1 before refine
## change dataset
## K = 13
f1 = 0
for round in range(1,6):
    print("Round : " + str(round))
    train, test = train_test_split(final3, test_size = 0.1)
    features = list(final3.columns)[7:]
    train_X = train[features]
    train_y = train['Target']
    test_X = test[features]
    test_y = test['Target']
    neigh = KNeighborsClassifier(n_neighbors = 13, weights='distance')
    neigh.fit(train_X, train_y)
    print('Training acc : ' + str(neigh.score(train_X, train_y)))
    print('Testing acc : ' + str(neigh.score(test_X, test_y)))
    pred = neigh.predict(test_X)
    curr_f1 = f1_score(list(test_y), pred, average='macro')
    f1 += curr_f1
    print('F1 : ' + str(curr_f1))
    print("= = = = = Round End = = = = =")
print("Average F1 : " + str(f1 * 1.0 / 5))

Round : 1
Training acc : 0.998485722527
Testing acc : 0.898767034393
F1 : 0.800797822777
= = = = = Round End = = = = =
Round : 2
Training acc : 0.998485722527
Testing acc : 0.904607397794
F1 : 0.811603073338
= = = = = Round End = = = = =
Round : 3
Training acc : 0.998557830978
Testing acc : 0.898767034393
F1 : 0.816856724765
= = = = = Round End = = = = =
Round : 4
Training acc : 0.998557830978
Testing acc : 0.89746917586
F1 : 0.798189260235
= = = = = Round End = = = = =
Round : 5
Training acc : 0.998557830978
Testing acc : 0.885139519792
F1 : 0.780719309762
= = = = = Round End = = = = =
Average F1 : 0.801633238175


In [39]:
def get_refined_df(test_combination, df):
    df_features = ['C123','C23','C12', 'C3','C2','C1']
    for i in range(len(df_features)):
        sub_df = df[df[df_features[i]].str.contains('^' + test_combination[i] + '$')]
        if sub_df.shape[0] > 0:
            # print("Pattern : " + df_features[i] + " , " + test_combination[i])
            return sub_df
    return df

In [37]:
## Get refined f1
def test_model(train, test, features):
    pred = []
    test_X_total = test[features]
    test_syllables_comb = test[syllables_comb]
    for i in range(test.shape[0]):
        curr_X = np.array(test_X_total.iloc[i]).reshape((1,-1))
        curr_syllables_comb = list(test_syllables_comb.iloc[i])
        
        sub_train = get_refined_df(curr_syllables_comb, train)
        train_X = sub_train[features]
        train_y = sub_train["Target"]
        
        if train_X.shape[0] < 13:
            neigh = KNeighborsClassifier(n_neighbors = train_X.shape[0], weights='distance')
            # neigh = KNeighborsClassifier(n_neighbors = 1)
        else:
            neigh = KNeighborsClassifier(n_neighbors = 13, weights='distance')
        neigh.fit(train_X, train_y)
        curr_pred = neigh.predict(curr_X)[0]
        pred.append(curr_pred)
    return pred

In [40]:
f1 = 0
for i in range(5):
    print("Round : " + str(i))
    train, test = train_test_split(final3, test_size = 0.1)
    features = list(final3.columns)[7:]
    syllables_comb = list(final3.columns)[1:7]
    test_y = list(test.Target)
    pred = test_model(train, test, features)
    curr_f1 = f1_score(test_y, pred, average='macro')
    f1 += curr_f1
    print('F1 : ' + str(curr_f1))
print('Average F1 : ' + str(f1 * 1.0 / 5))

Round : 0
F1 : 0.808890448141
Round : 1
F1 : 0.80864627997
Round : 2
F1 : 0.824242787949
Round : 3
F1 : 0.819109876826
Round : 4
F1 : 0.807649449412
Average F1 : 0.81370776846


## Vowel 2

In [41]:
## choose k
train, test = train_test_split(final2, test_size = 0.1)
features = list(final2.columns)[4:]
combinations = list(final2.columns)[1:4]
train_X = train[features]
train_y = train['Target']
test_X = test[features]
test_y = test['Target']
for i in range(5,50,5):
    print ('k = ' + str(i))
    neigh = KNeighborsClassifier(n_neighbors = i, weights='distance')
    neigh.fit(train_X, train_y)
    print('Training acc : ' + str(neigh.score(train_X, train_y)))
    print('Testing acc : ' + str(neigh.score(test_X, test_y)))
    pred = neigh.predict(test_X)
    print('F1 : ' + str(f1_score(list(test_y), pred, average='macro')))

k = 5
Training acc : 0.9971099426
Testing acc : 0.936800288913


/home/venturer/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1 : 0.575291562182
k = 10
Training acc : 0.9971099426
Testing acc : 0.936800288913
F1 : 0.572438897434
k = 15
Training acc : 0.9971099426
Testing acc : 0.940050559769
F1 : 0.574040439039
k = 20


KeyboardInterrupt: 

In [ ]:
## Get average f1 before refine
## change dataset
f1 = 0
for round in range(1,6):
    print("Round : " + str(round))
    train, test = train_test_split(final2, test_size = 0.1)
    features = list(final2.columns)[4:]
    train_X = train[features]
    train_y = train['Target']
    test_X = test[features]
    test_y = test['Target']
    neigh = KNeighborsClassifier(n_neighbors = 15, weights='distance')
    neigh.fit(train_X, train_y)
    print('Training acc : ' + str(neigh.score(train_X, train_y)))
    print('Testing acc : ' + str(neigh.score(test_X, test_y)))
    pred = neigh.predict(test_X)
    curr_f1 = f1_score(list(test_y), pred, average='macro')
    f1 += curr_f1
    print('F1 : ' + str(curr_f1))
    print("= = = = = Round End = = = = =")
print("Average F1 : " + str(f1 * 1.0 / 5))